In [271]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
import xgboost as xgb
import catboost as catb
import lightgbm as lgbmfrom
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
import xgboost as xgb
import catboost as catb
import lightgbm as lgbm
from sklearn.model_selection import GridSearchCV, cross_val_score, learning_curve
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from sklearn.base import clone
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from vecstack import stacking
from sklearn.ensemble import StackingClassifier

def make_predictions(filename, model):
    df_test = pd.read_csv('./data/test.csv')
    data_test = job(df_test)

    X_train_t, X_test_t, y_train_t, y_test_t, y_t, X_t, data_t, ids_t = get_data(False, data_test)

    y_predict =  model.predict(X_t)
    is_default = pd.DataFrame({'Id': ids_t,'Outcome':y_predict})

    is_default.to_csv(filename, index=False)

def get_classification_report(y_train_true, y_train_pred, y_test_true, y_test_pred):
    print('Train subset\n\n' + classification_report(y_train_true, y_train_pred))
    print('Test subset\n\n' + classification_report(y_test_true, y_test_pred))

def fit_and_show(clf, X_train, X_test, y_train, y_test):
    clf.fit(X_train, y_train)
    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)
    get_classification_report(y_train, y_train_pred, y_test, y_test_pred) 

def get_data(load, dt):
  
    if load:
        data = pd.read_pickle('work_data.pkl')
    else:
        data = dt

    target = 'Outcome'  
    if 'Outcome' in data.columns:
        y = data[target]
    else:
        y = data['Id']
        data[target] = data['Id']        

    scaler = StandardScaler()
    ids = data['Id']

    X = scaler.fit_transform(data.drop([target, 'Id'], axis=1))
    
    X_train, X_test, y_train, y_test = train_test_split( X, y, shuffle=True, test_size=0.3, random_state=42)

    return X_train, X_test, y_train, y_test, y, X, data, ids


def correct_value(data, target, value):
    data.loc[data[target] < value, target] = data[target].median()
    return data

# генераторы признаков квадраты и логорифмы
def create_poly_features(data, num_features):
    poly_features = np.power(data[num_features], 2)
    poly_features = poly_features.rename(columns=dict(zip(data.columns, [col+' sqr' for col in data.columns])))
    return data.join(poly_features)

def create_log_features(data, num_features):
    log_features = np.log(data[num_features], where=data[num_features]>0)
    log_features = log_features.rename(columns=dict(zip(data.columns, [col+'_log' for col in data.columns])))
    return data.join(log_features)

# Подготовим функцию обработки дата сета
def job(data):
      # Заменим тех у кого не измерили глюкозу на средние значения
      data = correct_value(data, 'Glucose', 60)
      # Установим средние значения для кровянного давления, у кого не измерили
      data = correct_value(data, 'BloodPressure', 60)
      # Установим средние значения тех у кого индекс массы тела меньше 18
      data = correct_value(data,'BMI', 18)
      # откорректируем толщину кожы в 5 мм так как этот фактор влияет на усвоение инсулина, и его нужно было измерять, тут явно пропуски
      data = correct_value(data,'SkinThickness', 5)
      # Откорректируем нижнюю границу нормы по инсулину
      data = correct_value(data,'Insulin', 15)

      #data=data.drop(['BloodPressure', 'SkinThickness', 'Insulin'], axis=1)
      
      # Добавим логарифмы и квадраты для числовых переменных
      data = create_poly_features(data, num_features)
      data = create_log_features(data, num_features)
      return data
num_features = ['Pregnancies', 'Glucose', 'BMI', 'Age'] # Insulin

In [272]:
X_train, X_test, y_train, y_test, y, X, data, ids = get_data(True, [])

In [273]:
base_learners2 = [
                ('rf_1', RandomForestClassifier(
                    max_depth=3,
                    min_samples_leaf=5,
                    criterion='entropy',
                    min_samples_split=5,
                    class_weight='balanced',
                    max_leaf_nodes=12,
                    random_state=43,
                    n_jobs=-1),)]               

In [274]:
lrs = LogisticRegression(multi_class='auto', solver='lbfgs', random_state=42)
sclf_st = StackingClassifier(estimators=base_learners2, final_estimator=lrs, cv=5)

In [275]:
fit_and_show(sclf_st, X_train, X_test, y_train, y_test)

Train subset

              precision    recall  f1-score   support

           0       0.84      0.88      0.86       252
           1       0.76      0.69      0.73       134

    accuracy                           0.82       386
   macro avg       0.80      0.79      0.80       386
weighted avg       0.82      0.82      0.82       386

Test subset

              precision    recall  f1-score   support

           0       0.77      0.87      0.82       102
           1       0.74      0.58      0.65        64

    accuracy                           0.76       166
   macro avg       0.75      0.73      0.73       166
weighted avg       0.76      0.76      0.75       166



In [276]:
make_predictions('predictions_stack_1.csv', sclf_st)